In [ ]:
#Python Codes for Moth Flame Optimization Algorithm
#step 1
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import random
import time
import pandas as pd
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
#step 2 loading data set
ds_tar1=pd.read_csv('d:/msk/jack data/target_all.csv') # Table 1: Target Dataset of 115 MFL Crack Images
ds_fea=pd.read_csv('d:/msk/jack data/all_features.csv') # Table 2: Features Dataset of 115 MFL Crack Images
ds_tar = ds_tar1.iloc[:,0] #for length change 1 to depth and 2 to width
#step 3 Moth Flame optimization algorithm 
fea=pd.DataFrame()
n_sample = 10 #number of Moths
nof = 17 #no. of features change this between 15 and 22
nitr = 10 #number of iterations
fnc1=['f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21','f22']#feature column names
fnc = fnc1[:nof]
trial = 'T1' #trail number
ps_tr = []
rs_tr = []
ps_tt = []
rs_tt = []
so = []
feano = pd.DataFrame()
mdl = XGBRegressor()
#population initialozation and calculation of objective function
for i in range(n_sample):
    print('Sample No.',i)
    fea[i]=pd.Series(random.sample(range(0, 22), nof)) 
feat = fea.T
for i in range(fea.shape[1]):
    for j in range(fea.shape[0]):
        feat.iloc[i,j] = fea.iloc[j,i]
#evaluation of population
for i in range(fea.shape[1]):
    ds_sfea = ds_fea.iloc[:,fea[i]]
    xtr, xtt, ytr, ytt = train_test_split(ds_sfea, ds_tar, test_size = 0.2, random_state = 2)
    mdl.fit(xtr, ytr)
    ytr_p = mdl.predict(xtr)
    ytt_p = mdl.predict(xtt)
    rs_tr.append(r2_score(ytr, ytr_p))
    ps_tr.append(np.sqrt(mean_squared_error(ytr, ytr_p)))
    rs_tt.append(r2_score(ytt, ytt_p))
    ps_tt.append(np.sqrt(mean_squared_error(ytt, ytt_p)))
    
pstr = pd.Series(ps_tr)
pstt = pd.Series(ps_tt)
rstr = pd.Series(rs_tr)
rstt = pd.Series(rs_tt)
per = pd.concat([pstr, pstt, rstr, rstt], axis=1)
per.columns=['pstr','pstt','rstr','rstt']
feat.columns=fnc
so = pd.concat([per, feat], axis = 1)
so['Obj']=so.iloc[:,0]*0.25 + so.iloc[:,1]*0.25 + so.iloc[:,2]*0.25 + so.iloc[:,3]*0.25
so[fnc]=so[fnc].astype('int64')
so1 = so
so1[fnc]=so1[fnc].astype('int64')
#selecting the best one
so1.sort_values('Obj', ascending = False)
bst = so1.iloc[0,:]
rslt = pd.DataFrame()
rs =[]
for i in range(bst.shape[0]):
    rs.append(bst[i])
rslt[0] = pd.Series(rs)
nso = so;
#initialization of GWO parameters
so_ad=so;#advanced
s1=[]
s2=[]
s3=[]
for it in range(nitr):
    print('Iteration ',it,'\nPopulation:',so)
    for i in range(n_sample):
        j1=round(random.random()*n_sample)-1
        while (j1==i):
            j1=round(random.random()*n_sample)-1
        SF=round(1+random.random())
        ri=round(n_sample*random.random())-1
        
        srt_pop=so.iloc[ri,4:-1]
        
        flno=round(n_sample-(it*(n_sample-1)/nitr))-1
        srt_pop1=so.iloc[flno,4:-1]
        
        rv=[]
        for ii in range(nof): rv.append(random.random())
        rvs=pd.Series(rv)
        so_newl=[]
        for ya in range(nof):
            so_newl.append(round(so.iloc[i,4+ya]+rvs[ya]*(srt_pop[ya]-SF*so.iloc[i,ya+4])))
        so_new=pd.Series(so_newl)
        for i1 in range(nof):
            so_new[i1]=round(so_new[i1])
        
        pos = so_new
        #print("so_new=",so_new)
        for j2 in range(nof):
            if pos[j] > 21:
                pos[j2]=21
            if pos[j2] < 0:
                pos[j2]=0
        
        npo = pd.Series(pos)      
        npo.columns = fnc              
        #npo[fnc]=npo[fnc].astype('int64')                   
        s1=[]
        s2=[]
        s3=[]
        s1=npo
        s2=[*set(s1)]               
        [s3.append(x) for x in list(range(22)) if x not in s2]
        len1 = nof - len(s2)
        for i1 in range(len1):
            s2.append(s3[i1])   
        
        npo=pd.Series(s2)        
        ds_sfean = ds_fea.iloc[:,s2]
        xtr1, xtt1, ytr1, ytt1 = train_test_split(ds_sfean, ds_tar, test_size = 0.2, random_state = 1)
        mdl.fit(xtr1, ytr1)
        ytr_p1 = mdl.predict(xtr1)
        ytt_p1 = mdl.predict(xtt1)
        rs_tr1=(r2_score(ytr1, ytr_p1))
        ps_tr1=(np.sqrt(mean_squared_error(ytr1, ytr_p1)))
        rs_tt1=(r2_score(ytt1, ytt_p1))
        ps_tt1=(np.sqrt(mean_squared_error(ytt1, ytt_p1)))
        obj1 = ps_tr1*0.25 + ps_tt1 *0.25 + rs_tr1*0.25 + rs_tt1 *0.25       
        nso.iloc[i,4:-1] = npo
        nper = pd.Series([ps_tr1,ps_tt1,rs_tr1,rs_tt1])
        nso.iloc[i,0:4] = nper
        nso.iloc[i,-1] = obj1
        dom_l=0
        dom_e=0
        dom_m=0
        for k in range(4):
            if nper[k]<so.iloc[i,k]:
                dom_l=dom_l+1
            elif nper[k]==so.iloc[i,k]:
                dom_e=dom_e+1
            else:
                dom_m=dom_m+1
        if dom_m==0 and dom_e!=4:
            so_ad.iloc[i,4:-1]=so.iloc[i,4:-1]
            so_ad.iloc[i,0:4]=so.iloc[i,0:4]
            so.iloc[i,4:-1]=npo
            so.iloc[i,0:4]=nper
        else:
            so_ad.iloc[i,4:-1]=npo
            so_ad.iloc[i,0:4]=nper
        dom_e=0
        dom_m=0
        for k in range(4):
            dom_m=dom_m+1
        if dom_m==0 and dom_e!=4:
            so_ad.iloc[j1,4:-1]=so.iloc[j1,4:-1]
            so_ad.iloc[j1,0:4]=so.iloc[j1,0:4]
        j1=round(random.random()*n_sample)-1
        while (j1==i):
            j1=round(random.random()*n_sample)-1
        for i1 in range(n_sample):
            a=-1+it*(-1/nitr)
            b=1
            t=(a-1)*random.random()+1
            if i1 <= flno:
                so_new=so.iloc[i1,4:-1]+abs(srt_pop-so.iloc[j1,4:-1]*math.exp(b*t)*math.cos(2*t*3.14))+srt_pop
            if i1>flno:
                so_new=so.iloc[i1,4:-1]+abs(srt_pop-so.iloc[j1,4:-1]*math.exp(b*t)*math.cos(2*t*3.14))+srt_pop1
        for i1 in range(nof):
            so_new[i1]=round(so_new[i1])
        pos = so_new
        for j2 in range(nof):
            if pos[j2] > 21:
                pos[j2]=21
            if pos[j2] < 0:
                pos[j2]=0
        npo = pd.Series(pos)      
        npo.columns = fnc              
        #npo[fnc]=npo[fnc].astype('int64')                   
        s1=[]
        s2=[]
        s3=[]
        s1=npo
        s2=[*set(s1)]               
        [s3.append(x) for x in list(range(22)) if x not in s2]
        len1 = nof - len(s2)
        for i1 in range(len1):
            s2.append(s3[i1])               
        npo=pd.Series(s2) 
        
        ds_sfean = ds_fea.iloc[:,s2]
        xtr1, xtt1, ytr1, ytt1 = train_test_split(ds_sfean, ds_tar, test_size = 0.2, random_state = 1)
        mdl.fit(xtr1, ytr1)
        ytr_p1 = mdl.predict(xtr1)
        ytt_p1 = mdl.predict(xtt1)
        rs_tr1=(r2_score(ytr1, ytr_p1))
        ps_tr1=(np.sqrt(mean_squared_error(ytr1, ytr_p1)))
        rs_tt1=(r2_score(ytt1, ytt_p1))
        ps_tt1=(np.sqrt(mean_squared_error(ytt1, ytt_p1)))
        obj1 = ps_tr1*0.25 + ps_tt1 *0.25 + rs_tr1*0.25 + rs_tt1 *0.25       
        nso.iloc[i,4:-1] = npo
        nper = pd.Series([ps_tr1,ps_tt1,rs_tr1,rs_tt1])
        dom_l=0
        dom_e=0
        dom_m=0
        for k in range(4):
            if nper[k]<so.iloc[i,k]:
                dom_l=dom_l+1
            elif nper[k]==so.iloc[i,k]:
                dom_e=dom_e+1
            else:
                dom_m=dom_m+1
        if dom_m==0 and dom_e!=4:
            so_ad.iloc[i,4:-1]=so.iloc[i,4:-1]
            so_ad.iloc[i,0:4]=so.iloc[i,0:4]
            so.iloc[i,4:-1]=npo
            so.iloc[i,0:4]=nper
        else:
            so_ad.iloc[i,4:-1]=npo
            so_ad.iloc[i,0:4]=nper
        j1=round(random.random()*n_sample)-1
        while (j1==i):
            j1=round(random.random()*n_sample)-1
        p_v=random.sample(range(0, 22),nof)
        npo=pd.Series(p_v)        
        ds_sfean = ds_fea.iloc[:,s2]
        xtr1, xtt1, ytr1, ytt1 = train_test_split(ds_sfean, ds_tar, test_size = 0.2, random_state = 1)
        mdl.fit(xtr1, ytr1)
        ytr_p1 = mdl.predict(xtr1)
        ytt_p1 = mdl.predict(xtt1)
        rs_tr1=(r2_score(ytr1, ytr_p1))
        ps_tr1=(np.sqrt(mean_squared_error(ytr1, ytr_p1)))
        rs_tt1=(r2_score(ytt1, ytt_p1))
        ps_tt1=(np.sqrt(mean_squared_error(ytt1, ytt_p1)))
        obj1 = ps_tr1*0.25 + ps_tt1 *0.25 + rs_tr1*0.25 + rs_tt1 *0.25       
        nso.iloc[i,4:-1] = npo
        nper = pd.Series([ps_tr1,ps_tt1,rs_tr1,rs_tt1])
        dom_l=0
        dom_e=0
        dom_m=0
        for k in range(4):
            if nper[k]<so.iloc[j1,k]:
                dom_l=dom_l+1
            elif nper[k]==so.iloc[j1,k]:
                dom_e=dom_e+1
            else:
                dom_m=dom_m+1
        if dom_m==0 and dom_e!=4:
            so_ad.iloc[j1,4:-1]=so.iloc[j1,4:-1]
            so_ad.iloc[j1,0:4]=so.iloc[j1,0:4]
            so.iloc[j1,4:-1]=npo
        else:
            so_ad.iloc[i,4:-1]=npo
            so_ad.iloc[i,0:4]=nper
    
    #combining so and so_ad
    cso_ad=pd.concat([so, so_ad], axis=0)
    cso1=cso_ad.sort_values('Obj',ascending=False)
    bst1=cso1.iloc[0,:]

    rs1=[]
    for i2 in range(bst1.shape[0]):
        rs1.append(bst1[i2])
    rslt[it+1] = pd.Series(rs1)
    
    so = cso1.iloc[0:n_sample,:]
####convergence plot
rslt1 = rslt.T
rslt1=pd.DataFrame(rslt1)
fna = 'd:/msk/JAck Data/mfo_lir_jack_nof' + str(nof) + '181222_25523.xlsx'
rslt1.to_excel(fna)
print(rslt1.shape[0])
ptr=[rslt1.iloc[0,0]]
ptt=[rslt1.iloc[0,1]]
rtr=[rslt1.iloc[0,2]]
rtt=[rslt1.iloc[0,3]]
sobj=[rslt1.iloc[0,-1]]
for i in range(rslt1.shape[0]):
    if rslt1.iloc[i,0] < ptr[i]:
        ptr.append(rslt1.iloc[i,0])
    else:
        ptr.append(ptr[i])
    if rslt1.iloc[i,1] < ptt[i]:
        ptt.append(rslt1.iloc[i,1])
    else:
        ptt.append(ptt[i])
    if rslt1.iloc[i,2] > rtr[i]:
        rtr.append(rslt1.iloc[i,2])
    else:
        rtr.append(rtr[i])
    if rslt1.iloc[i,3] > rtt[i]:
        rtt.append(rslt1.iloc[i,3])
    else:
        rtt.append(rtt[i])
    if rslt1.iloc[i,-1] > sobj[i]:
        sobj.append(rslt1.iloc[i,-1])
    else:
        sobj.append(sobj[i])        
x = range(len(ptr))
f1, ax1 = plt.subplots()
f2, ax2 = plt.subplots()
f3, ax3 = plt.subplots()
ax1.plot(x, ptr,'g-')
ax1.plot(x, ptt,'b-')
ax1.set_xlabel('Iteration Number')
ax1.set_ylabel('RMSE')
ax1.legend(['Training Dataset', 'Testing Dataset'])
ax2.plot(x, np.abs(rtr),'r-')
ax2.plot(x, np.abs(rtt),'m-')
ax2.set_xlabel('Iteration Number')
ax2.set_ylabel('${R{^2}}$')
ax2.legend(['Training Dataset', 'Testing Dataset'])    
ax3.plot(x, sobj, color='g')
ax3.set_xlabel('Iteration Number')
ax3.set_ylabel('Objective Value')
